# Download MARBERT checkpoint

In [25]:
!wget https://raw.githubusercontent.com/mramly/CairoMent/main/CairoDep_Datasets/CairoDep_Datasets/CairoDep_Datasets.csv

--2021-11-05 12:57:31--  https://raw.githubusercontent.com/mramly/CairoMent/main/CairoDep_Datasets/CairoDep_Datasets/CairoDep_Datasets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5758774 (5.5M) [text/plain]
Saving to: ‘CairoDep_Datasets.csv.2’

CairoDep_Datasets.c 100%[===================>]   5.49M  --.-KB/s    in 0.08s   

2021-11-05 12:57:31 (70.4 MB/s) - ‘CairoDep_Datasets.csv.2’ saved [5758774/5758774]



In [3]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert/

--2021-11-05 12:46:54--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 54.84.242.188, 50.19.215.42, 2600:1f18:147f:e800:edd:39e0:4b19:157c, ...
Connecting to huggingface.co (huggingface.co)|54.84.242.188|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc [following]
--2021-11-05 12:46:54--  https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 54.230.19.90, 54.230.19.89, 54.230.19.80, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|54.230.19.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607066087 (579M) [application/x-gzip]
Saving to: ‘MARBERT_pytorch_verison.tar.gz’

MARBERT_pytorch_ver 100%[==========

In [4]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [5]:
!pip install GPUtil pytorch_pretrained_bert transformers

     |████████████████████████████████| 123 kB 5.4 MB/s 
     |████████████████████████████████| 3.1 MB 36.6 MB/s 
     |████████████████████████████████| 131 kB 31.4 MB/s 
     |████████████████████████████████| 3.3 MB 27.1 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 8.1 MB 47.1 MB/s 
     |████████████████████████████████| 138 kB 42.9 MB/s 
     |████████████████████████████████| 127 kB 45.7 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=3602400091ad4849e8b5905f277749a0f57de529ed755b05fd5fb7138675515b
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    U

# Fine-tuning code

In [6]:
!pip install transformers==3.3.0

     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 1.2 MB 34.1 MB/s 
     |████████████████████████████████| 3.0 MB 35.6 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.3
    Uninstalling transformers-4.12.3:
      Successfully uninstalled transformers-4.12.3


In [7]:
# (1)load libraries 
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import ArabertPreprocessor


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [9]:
def preprocessing(df):
  model_name = "bert-base-arabertv2"
  arabert_prep = ArabertPreprocessor(model_name=model_name)
  df['post'] = df['post'].apply(lambda x: arabert_prep.preprocess(x))
  return df

In [10]:

def create_label2ind_file(file, label_col):
	#print(label_col)
	labels_json={}
	#load train_dev_test file
	df = pd.read_csv(file)
	df.head(5)
	#get labels and sort it A-Z
	labels = df[label_col].unique()
	labels.sort()
	#convert labels to indexes
	for idx in range(0, len(labels)):
		labels_json[labels[idx]]=idx
	#save labels with indexes to file
	with open(label2idx_file, 'w') as json_file:
		json.dump(labels_json, json_file)


In [11]:

def data_prepare_BERT(file_path, lab2ind, tokenizer, content_col, label_col, MAX_LEN):
	# Use pandas to load dataset
	df = pd.read_csv(file_path,header=0)
	df = df[df[content_col].notnull()]
	df = df[df[label_col].notnull()]
	print("Data size ", df.shape)
	# Create sentence and label lists
	preprocessing(df)
	sentences = df[content_col].values
	sentences = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences]
	print ("The first sentence:")
	print (sentences[0])
	# Create sentence and label lists
	labels = df[label_col].values
	#print (labels)
	print(lab2ind)
	labels = [lab2ind[i] for i in labels]
	# Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
	tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
	print ("Tokenize the first sentence:")
	print (tokenized_texts[0])
	#print("Label is ", labels[0])
	# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
	input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
	print ("Index numbers of the first sentence:")
	print (input_ids[0])
	# Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
	# ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
	pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
	input_ids = pad_sequences(input_ids, maxlen=MAX_LEN+2, dtype="long", truncating="post", padding="post", value=pad_ind)
	print ("Index numbers of the first sentence after padding:\n",input_ids[0])
	# Create attention masks
	attention_masks = []
	# Create a mask of 1s for each token followed by 0s for padding
	for seq in input_ids:
		seq_mask = [float(i > 0) for i in seq]
		attention_masks.append(seq_mask)
	# Convert all of our data into torch tensors, the required datatype for our model
	inputs = torch.tensor(input_ids)
	labels = torch.tensor(labels)
	masks = torch.tensor(attention_masks)
	return inputs, labels, masks


In [12]:
# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#	  pred_flat = np.argmax(preds, axis=1).flatten()
#	  labels_flat = labels.flatten()
#	  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_pred(preds, labels):
	pred_flat = np.argmax(preds, axis=1).flatten()
	labels_flat = labels.flatten()
	return pred_flat.tolist(), labels_flat.tolist()

In [13]:

def train(model, iterator, optimizer, scheduler, criterion):
	
	model.train()
	epoch_loss = 0
	for i, batch in enumerate(iterator):
		# Add batch to GPU
		batch = tuple(t.to(device) for t in batch)
		# Unpack the inputs from our dataloader
		input_ids, input_mask, labels = batch
		outputs = model(input_ids, input_mask, labels=labels)
		loss, logits = outputs[:2]
		# delete used variables to free GPU memory
		del batch, input_ids, input_mask, labels
		optimizer.zero_grad()
		if torch.cuda.device_count() == 1:
			loss.backward()
			epoch_loss += loss.cpu().item()
		else:
			loss.sum().backward()
			epoch_loss += loss.sum().cpu().item()
		optimizer.step()
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
		# optimizer.step()
		scheduler.step()
	# free GPU memory
	if device == 'cuda':
		torch.cuda.empty_cache()
	return epoch_loss / len(iterator)

In [14]:

def evaluate(model, iterator, criterion):
	model.eval()
	epoch_loss = 0
	all_pred=[]
	all_label = []
	with torch.no_grad():
		for i, batch in enumerate(iterator):
			# Add batch to GPU
			batch = tuple(t.to(device) for t in batch)
			# Unpack the inputs from our dataloader
			input_ids, input_mask, labels = batch
			outputs = model(input_ids, input_mask, labels=labels)
			loss, logits = outputs[:2]
			# delete used variables to free GPU memory
			del batch, input_ids, input_mask
			if torch.cuda.device_count() == 1:
				epoch_loss += loss.cpu().item()
			else:
				epoch_loss += loss.sum().cpu().item()
			# identify the predicted class for each example in the batch
			probabilities, predicted = torch.max(logits.cpu().data, 1)
			# put all the true labels and predictions to two lists
			all_pred.extend(predicted)
			all_label.extend(labels.cpu())
	accuracy = accuracy_score(all_label, all_pred)
	f1score = f1_score(all_label, all_pred, average='macro') 
	recall = recall_score(all_label, all_pred, average='macro')
	precision = precision_score(all_label, all_pred, average='macro')
	report = classification_report(all_label, all_pred)
	return (epoch_loss / len(iterator)), accuracy, f1score, recall, precision



In [15]:

def fine_tuning(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	# train_file = config["data_dir"]+config["train_file"]
	# dev_file = config["data_dir"]+config["dev_file"]
	train_file = config["train_file"]
	dev_file = config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoit directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.CrossEntropyLoss() as our loss function. 
	criterion = nn.CrossEntropyLoss()
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
		val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
# 		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df,model

# Run fine-tuning for 5 epochs

In [16]:

config={"task_name": "DEP_MARBERT", #output directory name
             "data_dir": "/content/drive/MyDrive/CairoDep_Datasets/CairoDep_Datasets", #data directory
             "train_file": "/content/train.csv", #train file path
             "dev_file": "/content/test.csv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 5, #number of epochs
             "content_col": "post", #text column
             "label_col": "label", #label column
             "lr": 2e-06, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 32, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [17]:
def data_spliter(df):
  train, test = train_test_split(df, test_size = 0.2,shuffle = True)
  train.to_csv('train.csv')
  test.to_csv('test.csv')

In [26]:
df = pd.read_csv('/content/CairoDep_Datasets.csv')
data_spliter(df)

In [ ]:
report_df,model= fine_tuning(config)

In [ ]:
report_df.head()